In [1]:
# import sys
# !{sys.executable} -m pip install -U matplotlib

In [2]:
import datetime, os, random, shutil, urllib.request, zipfile, time, warnings
warnings.filterwarnings('ignore')
from functools import wraps
from math import trunc
import numpy as np
import pandas as pd
from pathlib import Path
from urllib.request import urlopen
from zipfile import ZipFile
from scipy.sparse.linalg import norm
import scipy.sparse as ss
from scipy.sparse.linalg import svds
from tqdm import tqdm
from numba import njit
from sklearn.metrics import mean_squared_error as mse

SEED = 123
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
seed_everything()

# Preprocessing

In [3]:
def rmse(y_true, y_pred):
    return mse(y_true, y_pred, squared=False)

In [4]:
def get_dataset():
    path = Path("m1.zip")
    if not path.exists():
        with path.open("wb") as f:
            print("Downloading dataset...")
            f.write(urlopen("http://files.grouplens.org/datasets/movielens/ml-1m.zip").read())
    if not Path("ml-1m").is_dir():
        with ZipFile("m1.zip") as zf:
            zf.extractall()
    ratings_list = [i.strip().split("::") for i in open('ml-1m/ratings.dat', 'r').readlines()]
    ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
    ratings_df['Rating'] = ratings_df['Rating'].apply(pd.to_numeric)    
    return ratings_df

ratings_df = get_dataset()
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
SEED = 123
random.seed(SEED)

In [6]:
print("#users: " + str(ratings_df.UserID.nunique()))
print("#movies: " + str(ratings_df.MovieID.nunique()))
print("#ratings %: "+ str(ratings_df.shape[0] /\
                          (ratings_df.UserID.nunique()*ratings_df.MovieID.nunique()) * 100))

#users: 6040
#movies: 3706
#ratings %: 4.468362562231285


In [7]:
ratings_df.Rating.describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: Rating, dtype: float64

pivot ratings_df to get user-rating matrix format

In [8]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,10,100,1000,1002,1003,1004,1005,1006,1007,...,99,990,991,992,993,994,996,997,998,999
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


de-mean the data (normalize by each users mean) and convert it from a dataframe to a numpy array.

In [9]:
#R = R_df.as_matrix()
R = R_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

# Sample test data

create test data by sampling (and removing) 2,500 ratings

In [10]:
rated_indices = np.argwhere(R_demeaned>0)

In [11]:
test_set = random.sample(list(rated_indices), 2500)

In [12]:
print("1st index is "+str(test_set[0])+ " and its rating is "+str(R_demeaned[test_set[0][0], test_set[0][1]]))

1st index is [ 296 3087] and its rating is 4.705342687533729


In [13]:
test_ratings = []
for i in range(len(test_set)):
    test_ratings.append(R_demeaned[test_set[i][0],test_set[i][1]])
    R_demeaned[test_set[i][0], test_set[i][1]] = 0

In [14]:
print("1st index is " + str(test_set[0]) + " and its rating is " + str(R_demeaned[test_set[0][0], test_set[0][1]]))

1st index is [ 296 3087] and its rating is 0.0


# Singular Value Decomposition

Scipy function svds allow to choose the number of latent factors

In [15]:
U, sigma, Vt = svds(R_demeaned, k = 50)

convert it to the diagonal matrix form.

In [16]:
sigma = np.diag(sigma)

# Making Predictions from the Decomposed Matrices

can predict by simply multiplying the matrices to get the rank k=50 approximation of the ratings matrix.
(also need to add the user means back)

In [17]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(predicted_ratings, columns = R_df.columns)

In [18]:
pred_set = [max(predicted_ratings[x[0], x[1]],0) for x in test_set]

In [19]:
res_df = pd.DataFrame()

res_df["act"]=test_ratings
res_df["pred"]=pred_set
res_df.head(10)

,act,pred
0,4.705343,2.292930
1,2.844307,1.660000
2,0.058824,0.720654
3,4.817323,1.135902
4,1.869401,0.222807
5,3.722612,2.738209
6,1.586346,1.827266
7,4.871560,1.795515
8,2.588505,2.599185
9,0.942256,0.350657


finally, let's evaluate the RMSE:

In [20]:
np.sqrt(mse(test_ratings, pred_set))

2.196816569497359

# SVT

In [21]:
def get_train_set(data, indices):
    data = data.copy()
    data[indices[:,0],indices[:,1]] = 0
    return data

def normalize(data, mean):
    return data

def denormalize(data, mean):
    return data

def rmse(y_pred, y_true):
    return np.sqrt(np.mean((y_pred-y_true)**2))

In [22]:
# def SVT(M, iterations, func, tau, delta, debug=True):
#     Omega = M.nonzero()
#     tol = 0.001
#     incre = 5
    
#     # SVT
#     r = 0
#     P_Omega_M = ss.csr_matrix((np.ravel(M[Omega]), Omega), shape=M.shape)
#     normProjM = norm(P_Omega_M)
#     k0 = np.ceil(tau / (delta * normProjM))
#     Y = k0 * delta * P_Omega_M
    
#     bar = tqdm(range(iterations))
    
#     rmses = []
#     for k in bar:
#         s = r + 1
#         sparse_Y = ss.csc_matrix(Y)
#         u1, s1, v1 = sparsesvd(sparse_Y, s)
#         while np.min(s1) > tau and s >= min(*M.shape):
#             u1, s1, v1 = sparsesvd(sparse_Y, s)
#             s+=incre
        
#         r = np.sum(s1 > tau)
#         U = u1.T[:, :r]
#         V = v1[:r, :]
#         S = s1[:r] - tau
#         x = (U * S).dot(V)
        
#         x_omega = ss.csr_matrix((x[Omega], Omega), shape=M.shape)

#         reconstrcution_loss = norm(x_omega - P_Omega_M) / norm(P_Omega_M)
#         if reconstrcution_loss < tol:
#             break

#         diff = P_Omega_M - x_omega
#         Y += delta * diff

#         if k%5==0:
#             rmse_current = rmse(M[Omega], x[Omega])
#             test_rmse = func(x)
#             print('Iter %d , RMSE %.3f RMSE %.3f on test' % (k, rmse_current, test_rmse))
#             rmses.append(rmses)
#     return x, rmses

# test_true = R_df.values[test_indices[:,0],test_indices[:,1]]
# func = lambda X: rmse(np.clip(denormalize(X, user_ratings_mean)[test_indices[:,0],test_indices[:,1]], 1, 5), test_true)

# taus = [20000, 15000, 23000]
# deltas = [1.8, 2, 5]


# best = float('inf')
# best_model= None
# best_rmses = []

# results = []
# for tau, delta in itertools.product(taus,deltas):
#     print(tau, delta)
#     M, rmses  = SVT(train_set, 250, func, tau, delta), user_ratings_mean
#     model = np.clip(denormalize(M), 1,5)
#     test_pred = model[test_indices[:,0],test_indices[:,1]]
#     score = rmse(test_pred, test_true)
#     if score<best:
#         best = score
#         best_model = model
#         best_rmses = rmses
#     results.append({'tau': tau, 'delta': delta, 'rmse_test':score})
#     print(results[-1])

# test_pred = best_model[test_indices[:,0],test_indices[:,1]]

# sample_pred_df = pd.DataFrame({'test_pred': test_pred.ravel(), 'test_true': test_true.ravel()})
# print(sample_pred_df.head(10))

In [23]:
# PREPARE DATA FOR FUNK

user_ratings_mean = np.mean(R_df.values, axis = 1).reshape(-1,1)
R_demeaned = normalize(R_df.values, user_ratings_mean)

#split
np.random.seed(SEED)
random.seed(SEED)
test_indices = np.array(random.sample(list(np.argwhere(R_df.values>0)), 2500))
train_set = get_train_set(R_demeaned, test_indices)
train_set[R_df.values == 0] = 0

# Funk-SVD

# Funk Preprocessing

In [24]:
def fetch_ml_ratings(data_dir_path=None, verbose=False):
    if data_dir_path is None:
        data_dir_path = _get_data_dir_path(data_dir_path)
        dirname = 'ml-1m'
        filename = 'ratings.dat'
        csv_path = os.path.join(data_dir_path, dirname, filename)
        zip_path = os.path.join(data_dir_path, dirname) + '.zip'
        url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
    else:
        csv_path = data_dir_path

    if os.path.exists(csv_path):
        # Return data loaded into a DataFrame
        df = _ml_ratings_csv_to_df(csv_path)
        return df

    elif os.path.exists(zip_path):
        # Unzip file before calling back itself
        if verbose:
            print('Unzipping data...')
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(data_dir_path)
        os.remove(zip_path)
        return fetch_ml_ratings(verbose=verbose)

    else:
        # Download the ZIP file before calling back itself
        if verbose:
            print('Downloading data...')

        with urllib.request.urlopen(url) as r, open(zip_path, 'wb') as f:
            shutil.copyfileobj(r, f)
        return fetch_ml_ratings(verbose=verbose)

def _get_data_dir_path(data_dir_path=None):
    if data_dir_path is None:
        default = os.path.join('~', 'funk_svd_data')
        data_dir_path = os.environ.get('FUNK_SVD_DATA', default=default)
        data_dir_path = os.path.expanduser(data_dir_path)
    if not os.path.exists(data_dir_path):
        os.makedirs(data_dir_path)
    return data_dir_path

def _ml_ratings_csv_to_df(csv_path):
    names = ['u_id', 'i_id', 'rating', 'timestamp']
    dtype = {'u_id': np.uint32, 'i_id': np.uint32, 'rating': np.float64}
    def date_parser(time):
        return datetime.datetime.fromtimestamp(float(time))
    df = pd.read_csv(csv_path, names=names, dtype=dtype, header=0,
                     sep=r'::', parse_dates=['timestamp'],
                     date_parser=date_parser, engine='python')
    df.sort_values(by='timestamp', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Numba Methods

In [25]:
@njit
def _shuffle(X):
    np.random.shuffle(X)
    return X

@njit
def _initialization(n_users, n_items, n_factors):
    bu = np.zeros(n_users)
    bi = np.zeros(n_items)
    pu = np.random.normal(0, .1, (n_users, n_factors))
    qi = np.random.normal(0, .1, (n_items, n_factors))
    return bu, bi, pu, qi

@njit
def _run_epoch(X, bu, bi, pu, qi, global_mean, n_factors, lr, reg):
    for i in range(X.shape[0]):
        user, item, rating = int(X[i, 0]), int(X[i, 1]), X[i, 2]

        # Predict current rating
        pred = global_mean + bu[user] + bi[item]

        for factor in range(n_factors):
            pred += pu[user, factor] * qi[item, factor]
        err = rating - pred

        # Update biases
        bu[user] += lr * (err - reg * bu[user])
        bi[item] += lr * (err - reg * bi[item])

        # Update latent factors
        for factor in range(n_factors):
            puf = pu[user, factor]
            qif = qi[item, factor]
            pu[user, factor] += lr * (err * qif - reg * puf)
            qi[item, factor] += lr * (err * puf - reg * qif)
    return bu, bi, pu, qi

@njit
def _compute_val_metrics(X_val, bu, bi, pu, qi, global_mean, n_factors):
    residuals = []
    for i in range(X_val.shape[0]):
        user, item, rating = int(X_val[i, 0]), int(X_val[i, 1]), X_val[i, 2]
        pred = global_mean
        if user > -1:
            pred += bu[user]
        if item > -1:
            pred += bi[item]
        if (user > -1) and (item > -1):
            for factor in range(n_factors):
                pred += pu[user, factor] * qi[item, factor]
        residuals.append(rating - pred)
    residuals = np.array(residuals)
    loss = np.square(residuals).mean()
    rmse = np.sqrt(loss)
    mae = np.absolute(residuals).mean()
    return loss, rmse, mae

# Funk SVD Class

In [26]:
class SVD:
    def __init__(self, lr=.005, reg=.02, n_epochs=20, n_factors=100,
                 early_stopping=False, shuffle=False, min_delta=.001,
                 min_rating=1, max_rating=5):

        self.lr = lr
        self.reg = reg
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        self.early_stopping = early_stopping
        self.shuffle = shuffle
        self.min_delta = min_delta
        self.min_rating = min_rating
        self.max_rating = max_rating

    def fit(self, X, X_val=None):
        X = self._preprocess_data(X)

        if X_val is not None:
            X_val = self._preprocess_data(X_val, train=False, verbose=False)
            self._init_metrics()
        self.global_mean_ = np.mean(X[:, 2])
        self._run_sgd(X, X_val)
        return self

    def _preprocess_data(self, X, train=True, verbose=True):
#         print('Preprocessing data...\n')
        X = X.copy()

        if train:  # Mappings have to be created
            user_ids = X['u_id'].unique().tolist()
            item_ids = X['i_id'].unique().tolist()
            n_users = len(user_ids)
            n_items = len(item_ids)
            user_idx = range(n_users)
            item_idx = range(n_items)
            self.user_mapping_ = dict(zip(user_ids, user_idx))
            self.item_mapping_ = dict(zip(item_ids, item_idx))
        X['u_id'] = X['u_id'].map(self.user_mapping_)
        X['i_id'] = X['i_id'].map(self.item_mapping_)

        # Tag validation set unknown users/items with -1 (enables
        # `fast_methods._compute_val_metrics` detecting them)
        X.fillna(-1, inplace=True)
        
        X['u_id'] = X['u_id'].astype(np.int32)
        X['i_id'] = X['i_id'].astype(np.int32)
        return X[['u_id', 'i_id', 'rating']].values

    def _init_metrics(self):
        metrics = np.zeros((self.n_epochs, 3), dtype=np.float)
        self.metrics_ = pd.DataFrame(metrics, columns=['Loss', 'RMSE', 'MAE'])

    def _run_sgd(self, X, X_val):
        n_users = len(np.unique(X[:, 0]))
        n_items = len(np.unique(X[:, 1]))
        val_loss, val_rmse = '', ''
        bu, bi, pu, qi = _initialization(n_users, n_items, self.n_factors)

        # Run SGD
        pbar = tqdm(range(self.n_epochs), desc='Epoch',
                             ncols=110)
        for epoch_ix in pbar:
            pbar.set_postfix({'val_loss': val_loss, 'val_rmse': val_rmse})
            start = self._on_epoch_begin(epoch_ix)

            if self.shuffle:
                X = _shuffle(X)

            bu, bi, pu, qi = _run_epoch(X, bu, bi, pu, qi, self.global_mean_,
                                        self.n_factors, self.lr, self.reg)

            if X_val is not None:
                self.metrics_.loc[epoch_ix, :] = _compute_val_metrics(
                                                     X_val, bu, bi, pu, qi,
                                                     self.global_mean_,
                                                     self.n_factors)
                val_loss, val_rmse = self._on_epoch_end(start,
                                   self.metrics_.loc[epoch_ix, 'Loss'],
                                   self.metrics_.loc[epoch_ix, 'RMSE'],
                                   self.metrics_.loc[epoch_ix, 'MAE'])

                if self.early_stopping:
                    val_rmse = self.metrics_['RMSE'].tolist()
                    if self._early_stopping(val_rmse, epoch_ix,
                                            self.min_delta):
                        break
            else:
                val_loss, val_rmse = self._on_epoch_end(start)

        self.bu_ = bu
        self.bi_ = bi
        self.pu_ = pu
        self.qi_ = qi

    def predict(self, X, clip=True):
        return [
            self.predict_pair(u_id, i_id, clip)
            for u_id, i_id in zip(X['u_id'], X['i_id'])
        ]

    def predict_pair(self, u_id, i_id, clip=True):
        user_known, item_known = False, False
        pred = self.global_mean_

        if u_id in self.user_mapping_:
            user_known = True
            u_ix = self.user_mapping_[u_id]
            pred += self.bu_[u_ix]

        if i_id in self.item_mapping_:
            item_known = True
            i_ix = self.item_mapping_[i_id]
            pred += self.bi_[i_ix]

        if user_known and item_known:
            pred += np.dot(self.pu_[u_ix], self.qi_[i_ix])

        if clip:
            pred = self.max_rating if pred > self.max_rating else pred
            pred = self.min_rating if pred < self.min_rating else pred

        return pred

    def _early_stopping(self, val_rmse, epoch_idx, min_delta):
        if epoch_idx > 0:
            if val_rmse[epoch_idx] + min_delta > val_rmse[epoch_idx-1]:
                self.metrics_ = self.metrics_.loc[:(epoch_idx+1), :]
                return True
        return False

    def _on_epoch_begin(self, epoch_ix):
        start = time.time()
        end = '  | ' if epoch_ix < 9 else ' | '
#         print('Epoch {}/{}'.format(epoch_ix + 1, self.n_epochs), end=end)

        return start

    def _on_epoch_end(self, start, val_loss=None, val_rmse=None, val_mae=None):
        end = time.time()
        return f'{val_loss:.3f}', f'{val_rmse:.3f}'
#         print(f'took {end - start:.1f} sec')

In [27]:
ratings_df = get_dataset()
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)

In [28]:
seed_everything()
test_indices = np.array(random.sample(list(np.argwhere(R_df.values>0)), 2500))

In [29]:
df = fetch_ml_ratings().drop(columns=['timestamp'])

def split(df, R_df, test_indices):
    x_indices = test_indices[:,0]
    y_indices = test_indices[:,1]
    u_ids = R_df.index[x_indices].astype(int)
    i_ids = R_df.columns[y_indices].astype(int)

    test = []
    for u_id, i_id in zip(u_ids, i_ids):
        test.append(df.loc[(df['u_id'] == u_id) & (df['i_id'] == i_id)])
    test_df = pd.concat(test)
    train_df= df.drop(test_df.index.tolist())

    return train_df, test_df

train_df, test_df = split(df, R_df, test_indices)
test_df.head()

,u_id,i_id,rating
974970,1265,1732,4.0
540199,2456,2105,3.0
843159,1449,2959,4.0
98506,5350,2640,2.0
440052,3323,3468,4.0


In [30]:
def funk(train, test, **kw):
    hparams = '\n'.join(map(str, kw.items()))
    seed_everything()
    svd = SVD(**kw)
    svd.fit(X=train, X_val=test)
    y_true, y_pred = test['rating'], svd.predict(test)
    print(f"Test RMSE: {rmse(y_true, y_pred):.4f}\n\nHyperparams: \n{hparams}")

funk(train=train_df, test=test_df,
                shuffle=False, min_rating=1, max_rating=5, early_stopping=False,
                min_delta=.00001,
                lr=.001,
                reg=.002,
                n_epochs=100,
                n_factors=20)

Epoch: 100%|████████████████████████████████| 100/100 [00:07<00:00, 12.51it/s, val_loss=0.747, val_rmse=0.864]

Test RMSE: 0.8627

Hyperparams: 
('shuffle', False)
('min_rating', 1)
('max_rating', 5)
('early_stopping', False)
('min_delta', 1e-05)
('lr', 0.001)
('reg', 0.002)
('n_epochs', 100)
('n_factors', 20)


In [31]:
# Training took 16 sec
# Test RMSE: 0.8416

# Hyperparams: 
# ('shuffle', False)
# ('min_rating', 1)
# ('max_rating', 5)
# ('early_stopping', False)
# ('min_delta', 1e-05)
# ('lr', 0.0007)
# ('reg', 0.02)
# ('n_epochs', 300)
# ('n_factors', 20)

# Training took 20 sec
# Test RMSE: 0.8412

# Hyperparams: 
# ('shuffle', False)
# ('min_rating', 1)
# ('max_rating', 5)
# ('early_stopping', True)
# ('min_delta', 1e-05)
# ('lr', 0.0006)
# ('reg', 0.015)
# ('n_epochs', 1000)
# ('n_factors', 25)